<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Integrated_Framework_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# You can run this file directly in Google Colab:
# <a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Dynamic_MAS_rkAMM_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required libraries if in Colab
try:
    import google.colab
    print("Running in Colab, installing libraries...")
    !pip install requests pandas numpy matplotlib seaborn
except ImportError:
    print("Not running in Colab, skipping install.")


import numpy as np
import pandas as pd
import sys
import requests
import io
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
# --- AGENT DEFINITIONS (The Core Framework) ---
# ==============================================================================

class RiskAssessmentAgent:
    """ AGENT 1: The STATEFUL AI Agent (Simulated) """
    def __init__(self, initial_borrowers):
        # Initialize all borrowers with a "newer" profile
        self.borrower_reputations = {
            borrower_id: np.clip(np.random.normal(loc=45, scale=10), 10, 80)
            for borrower_id in initial_borrowers
        }
        # Constants for dynamic updates
        self.REPAY_BUMP = 3
        self.DEFAULT_PENALTY = -15
        self.MAX_COLLATERAL = 0.8
        self.MIN_P_D = 0.01
        self.MAX_P_D = 0.80

    def assess_loan_terms(self, borrower_id):
        """ Calculates PD and C based on the agent's current reputation """
        reputation = self.borrower_reputations.get(borrower_id, 45.0) # Default new borrowers
        p_d = self.MAX_P_D - (reputation / 100) * (self.MAX_P_D - self.MIN_P_D)
        collateral_level = self.MAX_COLLATERAL * (1.0 - (reputation / 100))
        return np.clip(p_d, self.MIN_P_D, self.MAX_P_D), np.clip(collateral_level, 0, self.MAX_COLLATERAL)

    def update_reputation(self, borrower_id, outcome):
        """ The reputation feedback loop """
        if outcome == "Repaid":
            self.borrower_reputations[borrower_id] += self.REPAY_BUMP
        else: # Default
            self.borrower_reputations[borrower_id] += self.DEFAULT_PENALTY
        self.borrower_reputations[borrower_id] = np.clip(
            self.borrower_reputations[borrower_id], 0, 100
        )

class PricingAgent:
    """ AGENT 2: The rkAMM Pricing Engine (On-Chain Logic) """
    def __init__(self):
        pass

    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):
        """ The Reverse Kelly Premium Formula (q) """
        L = loan_amount * (1.0 - collateral_level) # Loss Given Default
        # Handle edge cases
        if L <= 0 or p_d <= 0: return 0.0, L
        if p_d >= 1.0: return np.inf, L
        # Calculate premium
        premium = (L * p_d) / (1.0 - p_d)
        return premium, L

    def calculate_allocation_fraction(self, p_d, collateral_level, premium, loan_amount=1.0):
        """ The Kelly Allocation Formula (f*) """
        if premium <= 0 or loan_amount <= 0:
            return 0.0

        # q_ratio is the premium as a fraction of the amount at risk
        # L = loan_amount * (1-c)
        L = loan_amount * (1.0 - collateral_level)
        if L <= 0:
             # If fully collateralized, any premium is profit, allocation is "safe".
             # We cap at 1.0 (100% of pool) for safety, though f* would be infinite.
            return 1.0

        q_ratio = premium / L

        # f* = (q(1-p) - p) / q
        # This is a simplified form where 'p' is PD and 'q' is net odds
        numerator = (q_ratio * (1.0 - p_d)) - p_d

        if numerator <= 0:
            return 0.0 # Clip allocation, loan is unprofitable

        # The fraction f* is (numerator / q_ratio)
        return min(1.0, numerator / q_ratio)

class BlockchainEnforcementAgent:
    """ AGENT 3: The Blockchain Simulator """
    def __init__(self):
        pass

    def execute_loan(self, p_d):
        """ Simulates the binary outcome of the loan """
        return "Repaid" if np.random.rand() >= p_d else "Default"

# ==============================================================================
# --- SIMULATION A: Head-to-Head Ablation Study (for Table 1) ---
# ==============================================================================

def run_ablation_study(n_loans=10000, n_trials=50):
    print(f"--- Running Ablation Study (Head-to-Head, {n_trials} trials) ---")

    strategies = ["Reverse Kelly", "Proportional Premium", "Fixed Premium"]
    all_results = {s: [] for s in strategies}

    for trial in range(n_trials):
        sys.stdout.write(f"\r  Running trial {trial + 1}/{n_trials}...")
        sys.stdout.flush()

        # Generate a fixed stream of 10,000 loans for this trial
        loan_stream = []
        for _ in range(n_loans):
            loan_stream.append({
                'p_d': np.random.beta(2, 18), # Avg default rate ~10%
                'c': np.random.uniform(0.0, 0.6) # Collateral 0-60%
            })

        pricing_agent = PricingAgent()
        blockchain_agent = BlockchainEnforcementAgent()

        for strategy in strategies:
            pool_value = 10000.0 # Reset pool for each strategy
            capital_deployed = 0
            total_possible_capital = 0
            loan_amount = 100.0 # Fixed loan size
            drawdowns = []
            peak_value = pool_value
            losses = 0

            for loan in loan_stream:
                p_d, c = loan['p_d'], loan['c']
                premium, loss = 0.0, loan_amount * (1.0 - c)
                allocation_fraction = 0.0

                if strategy == "Reverse Kelly":
                    premium, _ = pricing_agent.calculate_premium(p_d, c, loan_amount)
                    allocation_fraction = pricing_agent.calculate_allocation_fraction(p_d, c, premium, loan_amount)

                elif strategy == "Proportional Premium":
                    premium = (2 * p_d * (1.0 - c)) * loan_amount # Premium = 2 * Expected Loss
                    allocation_fraction = 1.0 # Always deploy

                elif strategy == "Fixed Premium":
                    premium = (0.10 / 12) * loan_amount # 10% APR, monthly
                    allocation_fraction = 1.0 # Always deploy

                # Allocation is a fraction of the *pool*, but we cap at loan_amount
                # For this sim, we assume pool is large enough to fund if f* > 0
                allocation = 0.0
                if allocation_fraction > 0:
                    allocation = loan_amount # Fund the loan

                if allocation > 0 and pool_value >=